In [52]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [53]:
df = pd.read_csv('Resources/kc_house_data_test_features.csv', index_col=0) #holdout
final_model = pickle.load(open('poly2_modelx.pickle','rb'))

to_drop = ['id']
df = df.drop(['id'], axis=1)

In [54]:
print('Number of columns in model: ', len(final_model.coef_))

Number of columns in model:  28


In [55]:
df.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [56]:
# adjuting sqft home outliers to within 6 standard deviations of the mean
for col in ['sqft_above', 'sqft_living', 'sqft_living15']: 
    above_6std = df[col].mean()+(6*df[col].std())
    df[col] = np.where(df[col].values >above_6std, df[col].mean()+6*df[col].std(), df[col])
    
# adjusting lot outliers to within 9 standard deviations of the mean
for col in ['sqft_lot', 'sqft_lot15']: 
    above_9std = df[col].mean()+(9*df[col].std())
    df[col] = np.where(df[col].values >above_6std, df[col].mean()+9*df[col].std(), df[col])
    

In [57]:
#Fixing 'bathrooms' outliers
bathroom_median = df.bathrooms.median()
df.loc[df.bathrooms == 0, 'bathrooms'] = np.nan
df.fillna(bathroom_median,inplace=True)

In [58]:
df['month_sold'] = pd.to_datetime(df['date'].str.slice(0,8), format='%Y%m%d', errors='ignore').dt.month
df.drop('date',axis=1,inplace=True)

#adjusting bedrooms outliers 
df['bedrooms'] = df['bedrooms'].apply(lambda x : 9 if x > 9 else x)

In [60]:
# #Run model without this parameter first
# df['bathrooms'] = np.where(df.bathrooms>6, 6, df['bathrooms'])

In [61]:
## RUN THIS CELL ONLY ONCE!


# Create new column floors x sqft_living
df['floors_x_sqft_living'] = df['floors']*df['sqft_living']

#Create new column 'age' for age of house
df['age'] = 2015 - df.yr_built

#Create dummy variable 'basement' y/n
df['basement'] = np.where(df['sqft_basement']>0 , 1, 0)
df = pd.get_dummies(df, columns=['basement'], drop_first=True)

#Create a new column 'neighbors_compared' with sqft_living - sqft_living15
df['neighbors_compared'] = df.sqft_living - df.sqft_living15

#Create dummy columns of 'condition' 
df = pd.get_dummies(df, columns=['condition'], drop_first=True)

#Create dummy columns of 'floors'
df = pd.get_dummies(df, columns=['floors'], drop_first=True)


In [62]:
#RUN CELL ONLY ONCE

#Create new column 'last_renovted' for how long ago since last renovation
df['last_ren'] = np.where(df['yr_renovated'] > 0, (2015 - df['yr_renovated']), 0)

#Create new column 'last_ren2' for how long (years) since last renovation and including age if no renovation
df['last_ren2'] = np.where(df['yr_renovated'] > 0, (2015 - df['yr_renovated']), (2015 - df['yr_built']))

#Create dummy columns using 'renovated'
df['renovated'] = np.where(df['yr_renovated']>0 , 1, 0)
df = pd.get_dummies(df, columns=['renovated'], drop_first=True)

In [63]:
# Create dummy variables for zipcodes
zip_dummies = pd.get_dummies(df['zipcode'].astype(str), drop_first=True)
df.drop('zipcode',axis=1,inplace=True)

In [64]:
#concat zipcodes dataframe with main
df = pd.concat([df, zip_dummies], axis=1)

## Step 3: Predict the holdout set

In [65]:
df.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'waterfront',
       'view', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       ...
       '98146', '98148', '98155', '98166', '98168', '98177', '98178', '98188',
       '98198', '98199'],
      dtype='object', length=101)

In [66]:
poly= PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df)
poly_columns = poly.get_feature_names(df.columns)
df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [67]:
list(poly_columns)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'waterfront',
 'view',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'month_sold',
 'floors_x_sqft_living',
 'age',
 'basement_1',
 'neighbors_compared',
 'condition_2',
 'condition_3',
 'condition_4',
 'condition_5',
 'floors_1.5',
 'floors_2.0',
 'floors_2.5',
 'floors_3.0',
 'floors_3.5',
 'last_ren',
 'last_ren2',
 'renovated_1',
 '98002',
 '98003',
 '98004',
 '98005',
 '98006',
 '98007',
 '98008',
 '98010',
 '98011',
 '98014',
 '98019',
 '98022',
 '98023',
 '98024',
 '98027',
 '98028',
 '98029',
 '98030',
 '98031',
 '98032',
 '98033',
 '98034',
 '98038',
 '98039',
 '98040',
 '98042',
 '98045',
 '98052',
 '98053',
 '98055',
 '98056',
 '98058',
 '98059',
 '98065',
 '98070',
 '98072',
 '98074',
 '98075',
 '98077',
 '98092',
 '98102',
 '98103',
 '98105',
 '98106',
 '98107',
 '98108',
 '98109',
 '98112',
 '98115',
 '98116',
 '98117',
 '98118',
 '98119',

In [68]:
df_poly.shape

(4323, 5252)

In [69]:
df.shape

(4323, 101)

In [70]:
selected_columns =['sqft_living',
 'grade',
 'bedrooms sqft_living',
 'bathrooms sqft_living',
 'bathrooms grade',
 'bathrooms sqft_above',
 'bathrooms sqft_living15',
 'bathrooms floors_x_sqft_living',
 'sqft_living^2',
 'sqft_living grade',
 'sqft_living sqft_above',
 'sqft_living lat',
 'sqft_living long',
 'sqft_living sqft_living15',
 'sqft_living floors_x_sqft_living',
 'grade^2',
 'grade sqft_above',
 'grade lat',
 'grade long',
 'grade sqft_living15',
 'grade floors_x_sqft_living',
 'sqft_above^2',
 'sqft_above sqft_living15',
 'sqft_above floors_x_sqft_living',
 'lat floors_x_sqft_living',
 'long floors_x_sqft_living',
 'sqft_living15 floors_x_sqft_living',
 'floors_x_sqft_living^2']

In [71]:
final_pred = final_model.predict(df_poly[selected_columns])

In [72]:
final_pred.mean()

571284.7013751074

## Step 4: Export your predictions

In [73]:
last_df = pd.DataFrame(final_pred)

In [74]:
last_df

,0
0,670526.016548
1,670526.016548
2,420679.926140
3,466519.347429
4,546885.246498
...,...
4318,535399.005484
4319,563682.055923
4320,367848.096078
4321,403142.547094


In [75]:
last_df.to_csv("housing_preds_Candaur.csv'")